In [12]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib3
from requests import get
import time
import re
import os
import math

pd.options.display.max_rows = 8

In [8]:
#write url based on current time
url = "https://www.cmegroup.com/CmeWS/mvc/Quotes/Future/425/G?pageSize=50&_="
urlext = np.round(time.time(), decimals=0)
urlext = str(urlext)
url = url + urlext

# create a new Firefox session
driver = webdriver.Edge(executable_path=r'C:/Users/devas/Downloads/edgedriver_win64/msedgedriver.exe')
driver.get(url)
soup = BeautifulSoup(url)

#scrape site source page
urltext = soup.findAll(text=True)
urltext = driver.current_url
httptext = urllib3.PoolManager()
responsetext = httptext.request('GET', urltext)

#close browser
driver.close()

#convert BeautifulSoup object to text
souptext = BeautifulSoup(responsetext.data)
souptext2 = str(souptext)

<ipython-input-8-669837d84f0e>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path=r'C:/Users/devas/Downloads/edgedriver_win64/msedgedriver.exe')


In [9]:
souptext2

'<html><head>\n<title>Access Denied</title>\n</head><body>\n<h1>Access Denied</h1>\n \nYou don\'t have permission to access "http://www.cmegroup.com/CmeWS/mvc/Quotes/Future/425/G?" on this server.<p>\nReference #18.6de83217.1650548076.18e8599b\n</p></body>\n</html>\n'

In [15]:
#create lists for pricing and dates
settlePri = []
settleDate = []

#state the patterns to search on within raw data
pricetxt2 = '"priorSettle":"\d+.\d\d"'
pricetxt3 = '"priorSettle":"-"'

#find the above patterns in the text and append them to their corresponding lists from above
p = re.compile("(%s|%s)" % (pricetxt2, pricetxt3)).findall(souptext2) 
d = re.findall('"expirationDate":"\d{8}"', souptext2)
if p:
    settlePri.append(p)
if d:
    settleDate.append(d)

#combine those two lists
pricelist = {'Date':d,'Oil_Price':p}

#convert to dataframe
pricelist = pd.DataFrame(pricelist)

#remove unnecessary text, format numbers and dates, remove non-numeric price place holders from web site
pricelist['Date'] = pricelist['Date'].map(lambda x: x.replace('"expirationDate":"', ""))
pricelist['Date'] = [datetime(year=int(x[0:4]), month=int(x[4:6]), day=int(x[6:8])) for x in pricelist['Date']]
# pricelist['Month'] = pricelist['Date'].dt.month.astype(int)
# pricelist['Year'] = pricelist['Date'].dt.year.astype(int)
pricelist['Oil_Price'] = pricelist['Oil_Price'].map(lambda x: x.replace('"priorSettle":"', "").rstrip('"'))
pricelist['Oil_Price'] = pricelist['Oil_Price'].replace('-', '0.00')
pricelist['Oil_Price'] = pricelist['Oil_Price'].astype(float)

#final dataframe
# pricelist = pricelist[['Year', 'Month', 'Oil_Price']]
pricelist = pricelist[['Date', 'Oil_Price']]

In [16]:
pricelist

,Date,Oil_Price
